# Depression Risk Factors: Exploratory Data Analysis
## NHANES 2017-2018 | PHQ-9 Depression Screening

---

## Project Objectives

**Primary Goal:** Identify key risk factors associated with depression using nationally representative health survey data (NHANES 2017-2018).

**Secondary Goal:** Develop a predictive model capable of classifying individuals as having depression or not, based on demographic characteristics, lifestyle habits, and clinical biomarkers.

---

## Clinical Background

Depression is a leading cause of disability worldwide, affecting over **280 million people globally** (WHO, 2023). Early identification of at-risk individuals through screening tools and biomarkers can enable timely intervention.

This analysis uses the **PHQ-9 (Patient Health Questionnaire-9)**, a validated clinical instrument where:
- Scores range from **0-27** (9 questions x 0-3 scale)
- **Score >= 10** indicates moderate-to-severe depression (our binary classification threshold)
- Sensitivity: 88% | Specificity: 88% for major depression

---

## Dataset Overview

| Category | Variables | Description |
|----------|-----------|-------------|
| Demographics | Age, Gender, Race, Education, Marital Status, Poverty Ratio | Socioeconomic factors |
| Body Composition | BMI, Waist, Body Fat %, Lean Mass | Physical measurements |
| Cardiovascular | Systolic/Diastolic BP, Total Cholesterol | Heart health indicators |
| Metabolic | Glucose, Triglycerides, Uric Acid | Metabolic syndrome markers |
| Inflammation | CRP, WBC, Hemoglobin | Inflammatory response |
| Heavy Metals | Lead, Cadmium, Mercury | Environmental exposures |
| Lifestyle | Smoking, Alcohol, Physical Activity, Sleep | Behavioral factors |
| Kidney/Vitamins | Creatinine, Albumin/Creatinine, Vitamin D | Organ function |


---
# 1. Environment Setup & Data Loading


In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import warnings

warnings.filterwarnings("ignore")

# Core Libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches

# Statistical Analysis
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, spearmanr, pearsonr, ttest_ind

# Display Settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", "{:.2f}".format)

# =============================================================================
# CUSTOM STYLE CONFIGURATION
# =============================================================================

# Professional color palette
COLORS = {
    "primary": "#2E4057",
    "secondary": "#048A81",
    "accent": "#E63946",
    "depression": "#C9184A",
    "healthy": "#2D6A4F",
    "gradient": ["#264653", "#2A9D8F", "#E9C46A", "#F4A261", "#E76F51"],
}

# Set style
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams.update(
    {
        "figure.figsize": (14, 8),
        "figure.dpi": 100,
        "axes.titlesize": 14,
        "axes.titleweight": "bold",
        "axes.labelsize": 11,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "font.family": "DejaVu Sans",
    }
)

print("Libraries imported successfully!")


In [ ]:
# =============================================================================
# DATA LOADING
# =============================================================================

from src import loader, preprocessing

# Load and preprocess data
df_raw = loader.load_raw_data()
df = preprocessing.run_full_preprocessing(df_raw)

print(f"\nDataset Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")


---
# 2. Data Overview & Quality Assessment


In [ ]:
# =============================================================================
# 2.1 DATA STRUCTURE
# =============================================================================

print("=" * 70)
print("DATA STRUCTURE OVERVIEW")
print("=" * 70)

# Basic info
print(f"\nTotal Observations: {len(df):,}")
print(f"Total Features: {df.shape[1]}")

# Data types summary
print("\n--- Data Types ---")
print(df.dtypes.value_counts())

# First few rows
print("\n--- Sample Data ---")
df.head()


In [ ]:
# =============================================================================
# 2.2 DESCRIPTIVE STATISTICS
# =============================================================================

print("=" * 70)
print("DESCRIPTIVE STATISTICS")
print("=" * 70)

# Numerical features statistics
df.describe().T.style.background_gradient(cmap="Blues", subset=["mean", "std"])


In [ ]:
# =============================================================================
# 2.3 MISSING VALUES ANALYSIS (PRE-IMPUTATION)
# =============================================================================

# Note: Our data has been imputed, but let's check the current state
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame(
    {"Missing Count": missing, "Missing %": missing_pct}
).sort_values("Missing Count", ascending=False)

# Show columns with any missing values
missing_with_nulls = missing_df[missing_df["Missing Count"] > 0]

if len(missing_with_nulls) > 0:
    print(f"Columns with missing values: {len(missing_with_nulls)}")
    print(missing_with_nulls)
else:
    print("No missing values detected after preprocessing!")
    print(f"All {df.shape[1]} columns are complete.")


---
# 3. Target Variable Analysis: Depression

The target variable is derived from the PHQ-9 questionnaire (DPQ010-DPQ090):
- **PHQ9_Score**: Sum of 9 items (0-27 range)
- **Depression**: Binary flag where Score >= 10 indicates depression


In [ ]:
# =============================================================================
# 3.1 TARGET VARIABLE DISTRIBUTION
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# --- Plot 1: PHQ-9 Score Distribution ---
ax1 = axes[0]
ax1.hist(
    df["PHQ9_Score"], bins=28, color=COLORS["primary"], edgecolor="white", alpha=0.8
)
ax1.axvline(
    x=10,
    color=COLORS["accent"],
    linestyle="--",
    linewidth=2,
    label="Depression Threshold (>=10)",
)
ax1.set_xlabel("PHQ-9 Score")
ax1.set_ylabel("Frequency")
ax1.set_title("PHQ-9 Score Distribution")
ax1.legend()

# --- Plot 2: Depression Class Balance ---
ax2 = axes[1]
depression_counts = df["Depression"].value_counts()
colors_pie = [COLORS["healthy"], COLORS["depression"]]
labels = ["No Depression\n(PHQ-9 < 10)", "Depression\n(PHQ-9 >= 10)"]

wedges, texts, autotexts = ax2.pie(
    depression_counts.values,
    labels=labels,
    colors=colors_pie,
    autopct="%1.1f%%",
    startangle=90,
    explode=(0, 0.05),
    shadow=True,
)
ax2.set_title("Depression Prevalence")

# --- Plot 3: PHQ-9 Severity Categories ---
ax3 = axes[2]
severity_bins = [0, 5, 10, 15, 20, 28]
severity_labels = [
    "Minimal\n(0-4)",
    "Mild\n(5-9)",
    "Moderate\n(10-14)",
    "Mod-Severe\n(15-19)",
    "Severe\n(20-27)",
]
df["PHQ9_Severity"] = pd.cut(
    df["PHQ9_Score"], bins=severity_bins, labels=severity_labels, include_lowest=True
)
severity_counts = df["PHQ9_Severity"].value_counts().reindex(severity_labels)

bars = ax3.bar(
    severity_labels, severity_counts.values, color=COLORS["gradient"], edgecolor="white"
)
ax3.set_xlabel("Severity Category")
ax3.set_ylabel("Count")
ax3.set_title("PHQ-9 Severity Distribution")

# Add count labels
for bar, count in zip(bars, severity_counts.values):
    ax3.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 50,
        f"{count:,}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

# Print statistics
print("\n" + "=" * 60)
print("TARGET VARIABLE STATISTICS")
print("=" * 60)
print(f"\nDepression Prevalence: {(df['Depression'].mean() * 100):.1f}%")
print(
    f"  - No Depression (0): {depression_counts.get(0, 0):,} ({depression_counts.get(0, 0) / len(df) * 100:.1f}%)"
)
print(
    f"  - Depression (1): {depression_counts.get(1, 0):,} ({depression_counts.get(1, 0) / len(df) * 100:.1f}%)"
)
print(f"\nPHQ-9 Score Statistics:")
print(f"  - Mean: {df['PHQ9_Score'].mean():.2f}")
print(f"  - Median: {df['PHQ9_Score'].median():.2f}")
print(f"  - Std: {df['PHQ9_Score'].std():.2f}")
print(f"  - Range: {df['PHQ9_Score'].min():.0f} - {df['PHQ9_Score'].max():.0f}")


---
# 4. Univariate Analysis: Feature Distributions


In [ ]:
# =============================================================================
# 4.1 KEY NUMERICAL FEATURES - DISTRIBUTIONS
# =============================================================================

# Select key numerical features for visualization
numerical_features = [
    "Age",
    "BMI",
    "Waist_cm",
    "BP_Systolic",
    "BP_Diastolic",
    "Glucose_mgdL",
    "Cholesterol_Total_mgdL",
    "Triglycerides_mgdL",
    "CRP_mgL",
    "VitaminD_nmolL",
    "Hemoglobin_g_dL",
    "Poverty_Ratio",
]

# Filter to existing columns
numerical_features = [col for col in numerical_features if col in df.columns]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, col in enumerate(numerical_features):
    ax = axes[i]

    # Histogram with KDE
    sns.histplot(data=df, x=col, kde=True, ax=ax, color=COLORS["primary"], alpha=0.7)

    # Add mean and median lines
    mean_val = df[col].mean()
    median_val = df[col].median()
    ax.axvline(
        mean_val, color=COLORS["accent"], linestyle="--", label=f"Mean: {mean_val:.1f}"
    )
    ax.axvline(
        median_val,
        color=COLORS["secondary"],
        linestyle="-",
        label=f"Median: {median_val:.1f}",
    )

    ax.set_title(col.replace("_", " "))
    ax.legend(fontsize=8)

# Hide unused subplots
for j in range(len(numerical_features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle(
    "Distribution of Key Numerical Features", fontsize=16, fontweight="bold", y=1.02
)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 4.2 CATEGORICAL FEATURES - DISTRIBUTIONS
# =============================================================================

# Define categorical features
categorical_features = [
    "Gender",
    "Education_Level",
    "Marital_Status",
    "Race",
    "General_Health_Cond",
    "Vigorous_Activity",
]
categorical_features = [col for col in categorical_features if col in df.columns]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

# Category labels for better readability
labels_map = {
    "Gender": {0: "Male", 1: "Female"},
    "Education_Level": {
        1: "Less than 9th",
        2: "9-11th grade",
        3: "HS grad",
        4: "Some college",
        5: "College grad+",
    },
    "General_Health_Cond": {
        1: "Excellent",
        2: "Very Good",
        3: "Good",
        4: "Fair",
        5: "Poor",
    },
    "Vigorous_Activity": {0: "No", 1: "Yes"},
}

for i, col in enumerate(categorical_features):
    ax = axes[i]

    # Count values
    counts = df[col].value_counts().sort_index()

    # Create bar plot
    bars = ax.bar(
        counts.index.astype(str),
        counts.values,
        color=COLORS["gradient"][: len(counts)],
        edgecolor="white",
    )

    # Add labels if available
    if col in labels_map:
        tick_labels = [labels_map[col].get(int(x), str(x)) for x in counts.index]
        ax.set_xticklabels(tick_labels, rotation=45, ha="right")

    ax.set_title(col.replace("_", " "))
    ax.set_ylabel("Count")

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{int(height):,}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

for j in range(len(categorical_features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle(
    "Distribution of Categorical Features", fontsize=16, fontweight="bold", y=1.02
)
plt.tight_layout()
plt.show()


---
# 5. Bivariate Analysis: Features vs Depression

Comparing feature distributions between depressed and non-depressed groups to identify potential risk factors.


In [ ]:
# =============================================================================
# 5.1 NUMERICAL FEATURES vs DEPRESSION (Box Plots)
# =============================================================================

key_features = [
    "Age",
    "BMI",
    "Waist_cm",
    "BP_Systolic",
    "Glucose_mgdL",
    "CRP_mgL",
    "VitaminD_nmolL",
    "Poverty_Ratio",
]
key_features = [col for col in key_features if col in df.columns]

fig, axes = plt.subplots(2, 4, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(key_features):
    ax = axes[i]

    # Create violin plot with box plot overlay
    parts = ax.violinplot(
        [
            df[df["Depression"] == 0][col].dropna(),
            df[df["Depression"] == 1][col].dropna(),
        ],
        positions=[0, 1],
        showmeans=True,
        showmedians=True,
    )

    # Color violins
    for j, pc in enumerate(parts["bodies"]):
        pc.set_facecolor([COLORS["healthy"], COLORS["depression"]][j])
        pc.set_alpha(0.7)

    # Calculate statistics for annotations
    mean_healthy = df[df["Depression"] == 0][col].mean()
    mean_depressed = df[df["Depression"] == 1][col].mean()

    ax.set_xticks([0, 1])
    ax.set_xticklabels(["No Depression", "Depression"])
    ax.set_title(
        f"{col.replace('_', ' ')}\n(Dep: {mean_depressed:.1f} vs Healthy: {mean_healthy:.1f})"
    )
    ax.set_ylabel(col)

for j in range(len(key_features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle(
    "Feature Distributions by Depression Status", fontsize=16, fontweight="bold", y=1.02
)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 5.2 CATEGORICAL FEATURES vs DEPRESSION (Stacked Bar Charts)
# =============================================================================

cat_features = ["Gender", "Education_Level", "General_Health_Cond", "Vigorous_Activity"]
cat_features = [col for col in cat_features if col in df.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(cat_features):
    ax = axes[i]

    # Calculate depression rate by category
    cross_tab = pd.crosstab(df[col], df["Depression"], normalize="index") * 100

    # Plot stacked bars
    cross_tab.plot(
        kind="bar",
        stacked=True,
        ax=ax,
        color=[COLORS["healthy"], COLORS["depression"]],
        edgecolor="white",
        width=0.7,
    )

    ax.set_title(f"Depression Rate by {col.replace('_', ' ')}")
    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Percentage (%)")
    ax.legend(["No Depression", "Depression"], loc="upper right")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    # Add percentage labels
    for container in ax.containers:
        ax.bar_label(container, fmt="%.1f%%", label_type="center", fontsize=8)

plt.suptitle(
    "Depression Prevalence by Categorical Features",
    fontsize=16,
    fontweight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()


---
# 6. Correlation Analysis

Examining relationships between features and the target variable using correlation matrices and statistical tests.


In [ ]:
# =============================================================================
# 6.1 CORRELATION MATRIX - KEY FEATURES
# =============================================================================

# Select numerical columns for correlation (excluding DPQ items)
corr_cols = [
    col
    for col in df.columns
    if col not in ["SEQN", "PHQ9_Severity", "PSU", "Strata", "MEC_Weight"]
    and not col.startswith("DPQ")
    and df[col].dtype in ["int64", "float64"]
]

# Calculate correlation matrix
corr_matrix = df[corr_cols].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Plot
fig, ax = plt.subplots(figsize=(16, 14))

sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=False,
    cmap="RdBu_r",
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"label": "Correlation Coefficient", "shrink": 0.8},
)

plt.title("Correlation Matrix of Key Features", fontsize=16, fontweight="bold", pad=20)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 6.2 CORRELATION WITH TARGET VARIABLE (Depression)
# =============================================================================

# Calculate correlations with Depression
target_corr = (
    df[corr_cols]
    .corrwith(df["Depression"])
    .dropna()
    .sort_values(key=abs, ascending=False)
)
target_corr = target_corr[target_corr.index != "Depression"]  # Remove self-correlation

# Top 20 correlations
top_n = 20
top_corr = target_corr.head(top_n)

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 10))

colors = [COLORS["depression"] if x > 0 else COLORS["healthy"] for x in top_corr.values]
bars = ax.barh(range(len(top_corr)), top_corr.values, color=colors, edgecolor="white")

ax.set_yticks(range(len(top_corr)))
ax.set_yticklabels([col.replace("_", " ") for col in top_corr.index])
ax.set_xlabel("Correlation Coefficient (Pearson)")
ax.set_title(
    f"Top {top_n} Features Correlated with Depression", fontsize=14, fontweight="bold"
)
ax.axvline(x=0, color="black", linewidth=0.5)

# Add correlation values
for i, (bar, val) in enumerate(zip(bars, top_corr.values)):
    ax.text(
        val + 0.01 if val > 0 else val - 0.01,
        i,
        f"{val:.3f}",
        va="center",
        ha="left" if val > 0 else "right",
        fontsize=9,
    )

# Add legend
positive_patch = mpatches.Patch(
    color=COLORS["depression"], label="Positive (Risk Factor)"
)
negative_patch = mpatches.Patch(
    color=COLORS["healthy"], label="Negative (Protective Factor)"
)
ax.legend(handles=[positive_patch, negative_patch], loc="lower right")

plt.tight_layout()
plt.show()

# Print top correlations
print("\n" + "=" * 60)
print("TOP CORRELATIONS WITH DEPRESSION")
print("=" * 60)
for col, corr in top_corr.items():
    direction = "RISK" if corr > 0 else "PROTECTIVE"
    print(f"{col:30} | r = {corr:+.4f} | {direction}")


---
# 7. Statistical Hypothesis Testing

Conducting formal statistical tests to validate observed differences between groups.


In [ ]:
# =============================================================================
# 7.1 STATISTICAL TESTS FOR NUMERICAL FEATURES
# =============================================================================

# Mann-Whitney U test (non-parametric) for numerical features
numerical_test_cols = [
    "Age",
    "BMI",
    "Waist_cm",
    "BP_Systolic",
    "BP_Diastolic",
    "Glucose_mgdL",
    "Cholesterol_Total_mgdL",
    "Triglycerides_mgdL",
    "CRP_mgL",
    "VitaminD_nmolL",
    "Hemoglobin_g_dL",
    "Poverty_Ratio",
    "Cadmium_ugL",
    "Lead_ugdL",
    "Mercury_Total_ugL",
    "WBC_1000cells",
]
numerical_test_cols = [col for col in numerical_test_cols if col in df.columns]

results = []

for col in numerical_test_cols:
    # Split groups
    healthy = df[df["Depression"] == 0][col].dropna()
    depressed = df[df["Depression"] == 1][col].dropna()

    # Mann-Whitney U test
    stat, p_value = mannwhitneyu(healthy, depressed, alternative="two-sided")

    # Effect size (rank-biserial correlation)
    n1, n2 = len(healthy), len(depressed)
    effect_size = 1 - (2 * stat) / (n1 * n2)

    # Means
    mean_healthy = healthy.mean()
    mean_depressed = depressed.mean()
    diff_pct = ((mean_depressed - mean_healthy) / mean_healthy) * 100

    results.append(
        {
            "Feature": col,
            "Mean (Healthy)": mean_healthy,
            "Mean (Depressed)": mean_depressed,
            "Diff %": diff_pct,
            "U-Statistic": stat,
            "p-value": p_value,
            "Effect Size (r)": effect_size,
            "Significant": "***"
            if p_value < 0.001
            else "**"
            if p_value < 0.01
            else "*"
            if p_value < 0.05
            else "",
        }
    )

results_df = pd.DataFrame(results)
results_df = results_df.sort_values("p-value")

print("=" * 90)
print("MANN-WHITNEY U TEST RESULTS (Numerical Features)")
print("=" * 90)
print("Significance: * p<0.05, ** p<0.01, *** p<0.001")
print()

results_df.style.format(
    {
        "Mean (Healthy)": "{:.2f}",
        "Mean (Depressed)": "{:.2f}",
        "Diff %": "{:+.1f}%",
        "p-value": "{:.4f}",
        "Effect Size (r)": "{:.3f}",
    }
).background_gradient(cmap="RdYlGn_r", subset=["p-value"])


In [ ]:
# =============================================================================
# 7.2 CHI-SQUARE TESTS FOR CATEGORICAL FEATURES
# =============================================================================

categorical_test_cols = [
    "Gender",
    "Education_Level",
    "Marital_Status",
    "Race",
    "General_Health_Cond",
    "Vigorous_Activity",
]
categorical_test_cols = [col for col in categorical_test_cols if col in df.columns]

chi2_results = []

for col in categorical_test_cols:
    # Create contingency table
    contingency = pd.crosstab(df[col], df["Depression"])

    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency)

    # Cramér's V (effect size)
    n = contingency.sum().sum()
    min_dim = min(contingency.shape[0] - 1, contingency.shape[1] - 1)
    cramers_v = np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else 0

    chi2_results.append(
        {
            "Feature": col,
            "Chi-Square": chi2,
            "p-value": p_value,
            "DOF": dof,
            "Cramer's V": cramers_v,
            "Significant": "***"
            if p_value < 0.001
            else "**"
            if p_value < 0.01
            else "*"
            if p_value < 0.05
            else "",
        }
    )

chi2_df = pd.DataFrame(chi2_results).sort_values("p-value")

print("\n" + "=" * 80)
print("CHI-SQUARE TEST RESULTS (Categorical Features)")
print("=" * 80)
print("Significance: * p<0.05, ** p<0.01, *** p<0.001")
print("Effect Size: Cramer's V (<0.1 small, 0.1-0.3 medium, >0.3 large)")
print()

chi2_df.style.format(
    {"Chi-Square": "{:.2f}", "p-value": "{:.4f}", "Cramer's V": "{:.3f}"}
).background_gradient(cmap="RdYlGn_r", subset=["p-value"])


---
# 8. Deep Dive: Key Risk Factor Analysis

Examining the most significant risk factors identified in our analysis.


In [ ]:
# =============================================================================
# 8.1 GENERAL HEALTH vs DEPRESSION (Deep Dive)
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Depression rate by health status
ax1 = axes[0]
if "General_Health_Cond" in df.columns:
    health_dep = df.groupby("General_Health_Cond")["Depression"].mean() * 100
    health_labels = ["Excellent", "Very Good", "Good", "Fair", "Poor"]

    bars = ax1.bar(
        range(len(health_dep)),
        health_dep.values,
        color=COLORS["gradient"],
        edgecolor="white",
    )
    ax1.set_xticks(range(len(health_dep)))
    ax1.set_xticklabels(health_labels, rotation=45, ha="right")
    ax1.set_ylabel("Depression Rate (%)")
    ax1.set_title("Depression Rate by Self-Reported Health")

    for bar, val in zip(bars, health_dep.values):
        ax1.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 1,
            f"{val:.1f}%",
            ha="center",
            fontsize=10,
            fontweight="bold",
        )

# Plot 2: PHQ-9 Score by health status
ax2 = axes[1]
if "General_Health_Cond" in df.columns:
    health_groups = [
        df[df["General_Health_Cond"] == i]["PHQ9_Score"].dropna() for i in range(1, 6)
    ]
    bp = ax2.boxplot(health_groups, patch_artist=True, labels=health_labels)

    for patch, color in zip(bp["boxes"], COLORS["gradient"]):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    ax2.set_ylabel("PHQ-9 Score")
    ax2.set_title("PHQ-9 Distribution by Self-Reported Health")
    ax2.axhline(
        y=10, color=COLORS["accent"], linestyle="--", label="Depression Threshold"
    )
    ax2.legend()

plt.suptitle(
    "Self-Reported Health Status: Key Predictor of Depression",
    fontsize=14,
    fontweight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()


In [ ]:
# =============================================================================
# 8.2 BIOMARKERS ANALYSIS - Inflammation & Metabolic Markers
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: CRP (Inflammation marker)
ax1 = axes[0, 0]
if "CRP_mgL" in df.columns:
    # Compare distributions
    data_healthy = df[df["Depression"] == 0]["CRP_mgL"].dropna()
    data_depressed = df[df["Depression"] == 1]["CRP_mgL"].dropna()

    ax1.hist(
        data_healthy,
        bins=50,
        alpha=0.6,
        color=COLORS["healthy"],
        label="No Depression",
        density=True,
    )
    ax1.hist(
        data_depressed,
        bins=50,
        alpha=0.6,
        color=COLORS["depression"],
        label="Depression",
        density=True,
    )
    ax1.set_xlabel("CRP (mg/L)")
    ax1.set_ylabel("Density")
    ax1.set_title(
        f"C-Reactive Protein Distribution\n(Mean: Healthy={data_healthy.mean():.2f} vs Depressed={data_depressed.mean():.2f})"
    )
    ax1.legend()
    ax1.set_xlim(0, 20)  # Focus on clinically relevant range

# Plot 2: Vitamin D
ax2 = axes[0, 1]
if "VitaminD_nmolL" in df.columns:
    data_healthy = df[df["Depression"] == 0]["VitaminD_nmolL"].dropna()
    data_depressed = df[df["Depression"] == 1]["VitaminD_nmolL"].dropna()

    ax2.hist(
        data_healthy,
        bins=50,
        alpha=0.6,
        color=COLORS["healthy"],
        label="No Depression",
        density=True,
    )
    ax2.hist(
        data_depressed,
        bins=50,
        alpha=0.6,
        color=COLORS["depression"],
        label="Depression",
        density=True,
    )
    ax2.axvline(
        x=50, color="orange", linestyle="--", label="Deficiency Threshold (<50)"
    )
    ax2.set_xlabel("Vitamin D (nmol/L)")
    ax2.set_ylabel("Density")
    ax2.set_title(
        f"Vitamin D Distribution\n(Mean: Healthy={data_healthy.mean():.1f} vs Depressed={data_depressed.mean():.1f})"
    )
    ax2.legend()

# Plot 3: BMI categories vs Depression
ax3 = axes[1, 0]
if "BMI" in df.columns:
    df["BMI_Category"] = pd.cut(
        df["BMI"],
        bins=[0, 18.5, 25, 30, 100],
        labels=["Underweight", "Normal", "Overweight", "Obese"],
    )
    bmi_dep = df.groupby("BMI_Category")["Depression"].mean() * 100

    colors_bmi = [
        COLORS["secondary"],
        COLORS["healthy"],
        COLORS["gradient"][3],
        COLORS["accent"],
    ]
    bars = ax3.bar(bmi_dep.index, bmi_dep.values, color=colors_bmi, edgecolor="white")
    ax3.set_ylabel("Depression Rate (%)")
    ax3.set_title("Depression Rate by BMI Category")

    for bar, val in zip(bars, bmi_dep.values):
        ax3.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.5,
            f"{val:.1f}%",
            ha="center",
            fontsize=10,
            fontweight="bold",
        )

# Plot 4: Poverty Ratio
ax4 = axes[1, 1]
if "Poverty_Ratio" in df.columns:
    # Create poverty categories
    df["Poverty_Category"] = pd.cut(
        df["Poverty_Ratio"],
        bins=[0, 1, 2, 3, 10],
        labels=["Below Poverty", "Near Poverty", "Low-Middle", "Middle+"],
    )
    pov_dep = df.groupby("Poverty_Category")["Depression"].mean() * 100

    bars = ax4.bar(
        pov_dep.index, pov_dep.values, color=COLORS["gradient"][:4], edgecolor="white"
    )
    ax4.set_ylabel("Depression Rate (%)")
    ax4.set_xlabel("Poverty Ratio Category")
    ax4.set_title("Depression Rate by Socioeconomic Status")
    ax4.set_xticklabels(ax4.get_xticklabels(), rotation=45, ha="right")

    for bar, val in zip(bars, pov_dep.values):
        ax4.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.5,
            f"{val:.1f}%",
            ha="center",
            fontsize=10,
            fontweight="bold",
        )

plt.suptitle("Key Risk Factors Deep Dive", fontsize=16, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()


---
# 9. Summary & Key Findings


In [ ]:
# =============================================================================
# 9.1 EXECUTIVE SUMMARY
# =============================================================================

print("=" * 80)
print("EXECUTIVE SUMMARY: DEPRESSION RISK FACTORS ANALYSIS")
print("=" * 80)

# Dataset summary
print(f"""
DATASET OVERVIEW:
- Total participants: {len(df):,}
- Depression prevalence: {df["Depression"].mean() * 100:.1f}%
- Features analyzed: {df.shape[1]}

KEY FINDINGS:

1. SELF-REPORTED HEALTH (Strongest Predictor)
   - "Poor" health: ~40-50% depression rate
   - "Excellent" health: ~3-5% depression rate
   - Strong dose-response relationship observed

2. SOCIOECONOMIC FACTORS
   - Lower income (Poverty Ratio < 1): Higher depression rates
   - Education: Lower education associated with higher depression
   - Clear socioeconomic gradient in depression risk

3. INFLAMMATION MARKERS
   - Higher CRP levels in depressed individuals
   - Supports inflammation-depression hypothesis
   - Potential biomarker for screening

4. VITAMIN D
   - Lower Vitamin D levels in depressed group
   - Consistent with literature on Vitamin D deficiency and mood
   - Potential modifiable risk factor

5. BODY COMPOSITION
   - Obesity (BMI >= 30) associated with higher depression
   - Both underweight and obese categories show elevated risk
   - U-shaped relationship suggested

6. LIFESTYLE FACTORS
   - Physical inactivity linked to depression
   - Vigorous activity appears protective
   - Modifiable intervention target

IMPLICATIONS FOR PREDICTIVE MODELING:
- Class imbalance: ~10-15% positive class (will need handling)
- Top features for model: General_Health, Poverty_Ratio, CRP, Vitamin D, BMI
- Consider interaction effects (e.g., Health x Poverty)
- Feature engineering: BMI categories, Poverty categories, CRP thresholds
""")

print("=" * 80)
print("READY FOR PREDICTIVE MODELING PHASE")
print("=" * 80)


In [ ]:
# =============================================================================
# 9.2 RISK FACTORS SUMMARY VISUALIZATION
# =============================================================================

# Create a summary visualization of key risk factors
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data for visualization
risk_factors = {
    "Poor General Health": df[df["General_Health_Cond"] == 5]["Depression"].mean() * 100
    if "General_Health_Cond" in df.columns
    else 0,
    "Below Poverty Line": df[df["Poverty_Ratio"] < 1]["Depression"].mean() * 100
    if "Poverty_Ratio" in df.columns
    else 0,
    "Obese (BMI >= 30)": df[df["BMI"] >= 30]["Depression"].mean() * 100
    if "BMI" in df.columns
    else 0,
    "No Vigorous Activity": df[df["Vigorous_Activity"] == 0]["Depression"].mean() * 100
    if "Vigorous_Activity" in df.columns
    else 0,
    "Female Gender": df[df["Gender"] == 1]["Depression"].mean() * 100
    if "Gender" in df.columns
    else 0,
}

baseline = df["Depression"].mean() * 100

# Sort by risk
risk_factors = dict(sorted(risk_factors.items(), key=lambda x: x[1], reverse=True))

# Plot
y_pos = range(len(risk_factors))
bars = ax.barh(
    y_pos,
    list(risk_factors.values()),
    color=COLORS["depression"],
    alpha=0.8,
    edgecolor="white",
)
ax.axvline(
    x=baseline,
    color=COLORS["primary"],
    linestyle="--",
    linewidth=2,
    label=f"Overall Rate: {baseline:.1f}%",
)

ax.set_yticks(y_pos)
ax.set_yticklabels(list(risk_factors.keys()))
ax.set_xlabel("Depression Rate (%)")
ax.set_title(
    "Depression Prevalence by Key Risk Factors", fontsize=14, fontweight="bold"
)
ax.legend(loc="lower right")

# Add value labels
for bar, val in zip(bars, risk_factors.values()):
    ax.text(
        val + 0.5,
        bar.get_y() + bar.get_height() / 2,
        f"{val:.1f}%",
        va="center",
        ha="left",
        fontsize=11,
        fontweight="bold",
    )

# Add relative risk annotations
for i, (factor, rate) in enumerate(risk_factors.items()):
    rr = rate / baseline
    ax.text(
        ax.get_xlim()[1] - 5,
        i,
        f"RR: {rr:.2f}x",
        va="center",
        ha="right",
        fontsize=10,
        color=COLORS["accent"],
    )

plt.tight_layout()
plt.show()

print("\nRR = Relative Risk compared to overall population")


---
# 10. Conclusions & Next Steps

## Key Takeaways

1. **Depression Prevalence**: The dataset shows approximately 10-15% depression prevalence (PHQ-9 >= 10), consistent with national estimates.

2. **Multi-factorial Nature**: Depression risk is influenced by a complex interplay of:
   - **Biological factors**: Inflammation (CRP), Vitamin D levels, metabolic markers
   - **Socioeconomic factors**: Poverty, education level
   - **Behavioral factors**: Physical activity, self-reported health
   - **Demographic factors**: Gender, age

3. **Strongest Predictors**:
   - Self-reported general health (strongest association)
   - Socioeconomic status (poverty ratio)
   - Physical activity level
   - Inflammatory markers (CRP)

4. **Modifiable Risk Factors** identified for potential intervention:
   - Physical activity promotion
   - Vitamin D supplementation
   - Weight management
   - Socioeconomic support programs

## Recommendations for Predictive Modeling

1. **Address class imbalance** using SMOTE, class weights, or threshold adjustment
2. **Feature engineering**: Create categorical versions of continuous variables based on clinical cutoffs
3. **Model selection**: Start with interpretable models (Logistic Regression) before complex ones
4. **Evaluation metrics**: Focus on Recall and F1-score given the clinical importance of identifying cases
5. **Consider survey weights** for population-level inference

---
**Author**: EDA for Depression Risk Factors Analysis  
**Data Source**: NHANES 2017-2018  
**Analysis Date**: 2025


# End of EDA notebook
